In [13]:
# import data
import numpy as np
import pandas as pd

# create the training & test sets, skipping the header row with [1:]
dataset = pd.read_csv("../../tmp/train.csv")
target = dataset[[0]].values.ravel()
train = dataset.iloc[:,1:].values
test = pd.read_csv("../../tmp/test.csv").values

x_test = test.reshape((len(test),28*28))
x_train = train.reshape((len(train),28*28))[0:40000]
y_train = target[0:40000]

x_valid = train.reshape((len(train),28*28))[40000:]
y_valid = target[40000:]


In [14]:
# check data
print('x_train shape: {0}'.format(x_train.shape))
print('y_train shape: {0}'.format(y_train.shape))
print('x_test shape: {0}'.format(x_test.shape))

print('x_valid shape: {0}'.format(x_valid.shape))
print('y_valid shape: {0}'.format(y_valid.shape))


x_train shape: (40000, 784)
y_train shape: (40000,)
x_test shape: (28000, 784)
x_valid shape: (2000, 784)
y_valid shape: (2000,)


In [25]:
%%time

# model
import xgboost as xgb
import pandas as pd
import time 

params={
'booster':'gbtree',
# 这里手写数字是0-9，是一个多类的问题，因此采用了multisoft多分类器，
'objective': 'multi:softmax', 
'num_class':10, # 类数，与 multisoftmax 并用
'gamma':0.05,  # 在树的叶子节点下一个分区的最小损失，越大算法模型越保守 。[0:]
'max_depth':12, # 构建树的深度 [1:]
#'lambda':450,  # L2 正则项权重
'subsample':0.4, # 采样训练数据，设置为0.5，随机选择一般的数据实例 (0:1]
'colsample_bytree':0.7, # 构建树树时的采样比率 (0:1]
#'min_child_weight':12, # 节点的最少特征数
'silent':1 ,
'eta': 0.005, # 学习率
'seed':710,
'nthread':6,# cpu 线程数
}

plst = list(params.items())
num_rounds = 600 # 迭代次数, 最好选500

xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(x_train, label=y_train)
xgval = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(xgtrain, 'train'),(xgval, 'val')]

history = model = xgb.train(plst, xgtrain, num_rounds, watchlist)# , early_stopping_rounds=100


[0]	train-merror:0.106325	val-merror:0.144000
[1]	train-merror:0.069200	val-merror:0.116000
[2]	train-merror:0.057600	val-merror:0.094500
[3]	train-merror:0.051100	val-merror:0.091000
[4]	train-merror:0.046700	val-merror:0.083500
[5]	train-merror:0.045050	val-merror:0.080000
[6]	train-merror:0.043475	val-merror:0.081000
[7]	train-merror:0.041900	val-merror:0.076500
[8]	train-merror:0.040950	val-merror:0.074000
[9]	train-merror:0.040850	val-merror:0.074000
[10]	train-merror:0.039950	val-merror:0.071500
[11]	train-merror:0.039100	val-merror:0.071500
[12]	train-merror:0.038500	val-merror:0.069500
[13]	train-merror:0.038425	val-merror:0.069500
[14]	train-merror:0.037775	val-merror:0.069000
[15]	train-merror:0.037475	val-merror:0.071000
[16]	train-merror:0.037225	val-merror:0.069500
[17]	train-merror:0.036825	val-merror:0.070500
[18]	train-merror:0.036575	val-merror:0.073000
[19]	train-merror:0.036150	val-merror:0.073000
[20]	train-merror:0.035825	val-merror:0.073500
[21]	train-merror:0.035

CPU times: user 1h 54min 54s, sys: 1.22 s, total: 1h 54min 55s
Wall time: 19min 11s


[599]	train-merror:0.009400	val-merror:0.048500


In [26]:
model.save_model('xgb.model') # 用于存储训练出的模型

preds = model.predict(xgtest,ntree_limit=model.best_iteration)

# 将预测结果写入文件
np.savetxt('submission_xgb_MultiSoftmax.csv',np.c_[range(1,len(test)+1),preds],
                delimiter=',',header='ImageId,Label',comments='',fmt='%d')


# kaggle result:  0.95414